In [ ]:
pip install numpy pandas scikit-learn tensorflow gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 MB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.1/322.1 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 80.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 3.5 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.5.1+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
to

In [2]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import gradio as gr
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from scipy.stats import linregress
import os
import datetime
import hashlib
import json

# Generate synthetic data with 7-day measurements
np.random.seed(42)
n_samples = 30000
n_days = 7

# Database file path
DATABASE_FILE = "patient_database.csv"
USER_FILE = "users.json"

# User management functions
def initialize_users():
    if not os.path.exists(USER_FILE):
        default_users = {
            "admin": {
                "password": hashlib.sha256("admin123".encode()).hexdigest(),
                "role": "admin"
            },
            "doctor": {
                "password": hashlib.sha256("doctor123".encode()).hexdigest(),
                "role": "doctor"
            }
        }
        with open(USER_FILE, 'w') as f:
            json.dump(default_users, f)

def validate_user(username, password):
    if not os.path.exists(USER_FILE):
        initialize_users()

    with open(USER_FILE, 'r') as f:
        users = json.load(f)

    if username in users:
        hashed_password = hashlib.sha256(password.encode()).hexdigest()
        if users[username]["password"] == hashed_password:
            return True, users[username]["role"]

    return False, None

def add_user(username, password, role):
    if not os.path.exists(USER_FILE):
        initialize_users()

    with open(USER_FILE, 'r') as f:
        users = json.load(f)

    if username in users:
        return False, "Username already exists"

    users[username] = {
        "password": hashlib.sha256(password.encode()).hexdigest(),
        "role": role
    }

    with open(USER_FILE, 'w') as f:
        json.dump(users, f)

    return True, "User added successfully"

# Generate features with temporal patterns
def generate_temporal(base, trend, noise=0.1):
    return [base + trend*d + np.random.normal(0, noise) for d in range(n_days)]

data = []
for _ in range(n_samples):
    age = np.random.randint(18, 85)
    gender = np.random.choice(['Male', 'Female'])
    smoking = np.random.choice(['Never', 'Former', 'Current'], p=[0.5, 0.3, 0.2])

    health_status = np.random.choice(['Healthy', 'Disease'], p=[0.7, 0.3])
    base_fev1 = np.random.normal(3.5, 0.5) if health_status == 'Healthy' else np.random.normal(2.5, 0.4)
    base_fvc = np.random.normal(4.0, 0.4)
    base_tlc = np.random.normal(5.0, 0.5) if health_status == 'Healthy' else np.random.normal(6.5, 0.6)
    base_vc = np.random.normal(4.5, 0.5) if health_status == 'Healthy' else np.random.normal(3.5, 0.4)

    fev1_trend = 0 if health_status == 'Healthy' else -0.05
    fvc_trend = 0 if health_status == 'Healthy' else -0.02
    tlc_trend = 0 if health_status == 'Healthy' else 0.03
    vc_trend = 0 if health_status == 'Healthy' else -0.03

    fev1_values = generate_temporal(base_fev1, fev1_trend)
    fvc_values = generate_temporal(base_fvc, fvc_trend)
    tlc_values = generate_temporal(base_tlc, tlc_trend)
    vc_values = generate_temporal(base_vc, vc_trend)

    ratios = [fev1/fvc for fev1, fvc in zip(fev1_values, fvc_values)]

    agg_features = {
        'age': age,
        'gender': gender,
        'smoking': smoking,
        'fev1_mean': np.mean(fev1_values),
        'fvc_mean': np.mean(fvc_values),
        'tlc_mean': np.mean(tlc_values),
        'vc_mean': np.mean(vc_values),
        'ratio_mean': np.mean(ratios),
        'ratio_min': np.min(ratios),
        'ratio_slope': linregress(range(7), ratios).slope,
        'disease': 1 if health_status == 'Disease' else 0
    }
    data.append(agg_features)

df = pd.DataFrame(data)

# Preprocessing
preprocessor = ColumnTransformer([
    ('num', StandardScaler(), ['age', 'fev1_mean', 'fvc_mean', 'tlc_mean', 'vc_mean',
                              'ratio_mean', 'ratio_min', 'ratio_slope']),
    ('cat', OneHotEncoder(), ['gender', 'smoking'])
])

X = df.drop('disease', axis=1)
y = df['disease']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train_pre = preprocessor.fit_transform(X_train)
X_test_pre = preprocessor.transform(X_test)

# Model Architecture
model = Sequential([
    Dense(128, activation='relu', input_shape=(X_train_pre.shape[1],)),
    BatchNormalization(),
    Dropout(0.3),
    Dense(64, activation='relu'),
    BatchNormalization(),
    Dropout(0.2),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy', tf.keras.metrics.AUC(name='auc')])

history = model.fit(X_train_pre, y_train,
                    epochs=50,
                    batch_size=128,
                    validation_split=0.2,
                    callbacks=[tf.keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True)])

# Database Functions
def initialize_database():
    if not os.path.exists(DATABASE_FILE):
        columns = ['timestamp', 'patient_id', 'age', 'gender', 'smoking',
                   'fev1_mean', 'fvc_mean', 'tlc_mean', 'vc_mean',
                   'ratio_mean', 'ratio_min', 'ratio_slope',
                   'prediction', 'confidence']
        pd.DataFrame(columns=columns).to_csv(DATABASE_FILE, index=False)

def save_to_database(patient_data):
    if not os.path.exists(DATABASE_FILE):
        initialize_database()
    db = pd.read_csv(DATABASE_FILE)
    db = pd.concat([db, pd.DataFrame([patient_data])], ignore_index=True)
    db.to_csv(DATABASE_FILE, index=False)
    return f"Data saved successfully. Total records: {len(db)}"

def load_database():
    if not os.path.exists(DATABASE_FILE):
        initialize_database()
        return pd.DataFrame()
    return pd.read_csv(DATABASE_FILE)

# Prediction Function
def calculate_ratios(fev1_values, fvc_values):
    return [fev1/fvc for fev1, fvc in zip(fev1_values, fvc_values)]

def predict_health(age, gender, smoking, patient_id, *daily_measurements):
    measurements = list(daily_measurements)
    fev1_values, fvc_values, tlc_values, vc_values = [], [], [], []

    for i in range(0, len(measurements), 4):
        if i+3 < len(measurements):
            fev1_values.append(measurements[i])
            fvc_values.append(measurements[i+1])
            tlc_values.append(measurements[i+2])
            vc_values.append(measurements[i+3])

    ratios = calculate_ratios(fev1_values, fvc_values)
    days = list(range(1, len(ratios) + 1))

    plt.style.use('dark_background')
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))

    ax1.plot(days, ratios, marker='o', color='#E50914')
    ax1.set_title('FEV1/FVC Ratio Trend', color='white')
    ax1.grid(color='#404040')

    params = ['FEV1', 'FVC', 'TLC', 'VC']
    values = [np.mean(fev1_values), np.mean(fvc_values),
             np.mean(tlc_values), np.mean(vc_values)]
    ax2.bar(params, values, color=['#E50914', '#B81D24', '#221F1F', '#F5F5F1'])
    ax2.set_title('Average Parameters', color='white')
    plt.tight_layout()

    fev1_mean = np.mean(fev1_values)
    fvc_mean = np.mean(fvc_values)
    tlc_mean = np.mean(tlc_values)
    vc_mean = np.mean(vc_values)
    ratio_mean = np.mean(ratios)
    ratio_min = min(ratios)
    ratio_slope = linregress(days, ratios).slope

    input_data = pd.DataFrame([{
        'age': age,
        'gender': gender,
        'smoking': smoking,
        'fev1_mean': fev1_mean,
        'fvc_mean': fvc_mean,
        'tlc_mean': tlc_mean,
        'vc_mean': vc_mean,
        'ratio_mean': ratio_mean,
        'ratio_min': ratio_min,
        'ratio_slope': ratio_slope
    }])

    processed_input = preprocessor.transform(input_data)
    prediction = model.predict(processed_input, verbose=0)[0][0]
    confidence = max(prediction, 1-prediction)
    prediction_label = 'At Risk' if prediction >= 0.5 else 'Healthy'

    timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    patient_data = {
        'timestamp': timestamp,
        'patient_id': patient_id,
        'age': age,
        'gender': gender,
        'smoking': smoking,
        'fev1_mean': fev1_mean,
        'fvc_mean': fvc_mean,
        'tlc_mean': tlc_mean,
        'vc_mean': vc_mean,
        'ratio_mean': ratio_mean,
        'ratio_min': ratio_min,
        'ratio_slope': ratio_slope,
        'prediction': prediction_label,
        'confidence': confidence
    }

    save_to_database(patient_data)

    report = f"""
    ## 📋 Medical Report
    **Patient ID:** {patient_id}
    **Age:** {age}
    **Gender:** {gender}
    **Smoking Status:** {smoking}

    **Key Indicators:**
    🔴 Average FEV1/FVC Ratio: {ratio_mean:.2f}
    🔴 Minimum Ratio: {ratio_min:.2f}
    🔴 TLC: {tlc_mean:.1f} L
    🔴 VC: {vc_mean:.1f} L

    **Prediction:** {'🟢 Low Risk (Healthy)' if prediction < 0.5 else '🔴 High Risk (Potential Lung Disease)'}
    **Confidence:** {confidence:.1%}

    **Record Status:** Saved to database at {timestamp}
    """

    return fig, report, {"Healthy": 1-prediction, "At Risk": prediction}

def view_database():
    db = load_database()
    if len(db) == 0:
        return "No records found in the database."

    db['timestamp'] = pd.to_datetime(db['timestamp'])
    db['date'] = db['timestamp'].dt.date
    db['time'] = db['timestamp'].dt.time
    display_cols = ['date', 'time', 'patient_id', 'age', 'gender', 'prediction', 'confidence']

    if 'confidence' in db.columns:
        db['confidence'] = (db['confidence'] * 100).round(1).astype(str) + '%'

    return db[display_cols].to_html(index=False, classes="data-table") if len(db) > 0 else "No records found."

# Login Function
def login(username, password):
    success, role = validate_user(username, password)
    if success:
        return gr.update(visible=False), gr.update(visible=True), f"Welcome, {username}! Role: {role}"
    else:
        return gr.update(visible=True), gr.update(visible=False), "Invalid username or password. Please try again."

def handle_logout():
    return gr.update(visible=True), gr.update(visible=False), "Successfully logged out. Please login again."

# CSS Styling
css = """
/* Previous CSS styles remain the same */
/* Add logout button positioning */
.logout-container {
    position: fixed;
    bottom: 20px;
    left: 20px;
    z-index: 1000;
}
"""

theme = gr.themes.Default(
    primary_hue="green",
    secondary_hue="blue",
    neutral_hue="slate",
    font=[gr.themes.GoogleFont("Inter")]
)

initialize_database()
initialize_users()

with gr.Blocks(css=css, theme=theme) as demo:
    gr.Markdown('<div class="water-effect"></div>')

    # Login Interface
    with gr.Group(elem_id="login-block") as login_block:
        gr.Markdown('<div class="login-container fade-in">')
        gr.Markdown('<div class="login-logo"><h1>🫁 Lung Health Analyzer</h1></div>')
        gr.Markdown('<h3 style="text-align: center; margin-bottom: 30px;">Secure Login</h3>')
        username = gr.Textbox(label="Username", placeholder="Enter your username", elem_classes=["login-input"])
        password = gr.Textbox(label="Password", placeholder="Enter your password", type="password", elem_classes=["login-input"])
        login_button = gr.Button("Sign In", elem_classes=["login-button", "ripple"])
        login_message = gr.Markdown("")
        gr.Markdown('</div>')

    # Main Application Interface
    with gr.Group(visible=False) as main_app:
        gr.Markdown('# 🏥 Lung Health Analyzer')
        logout_btn = gr.Button("🚪 Logout", elem_classes=["logout-button", "ripple"])

        with gr.Tabs():
            with gr.Tab("Patient Analysis"):
                with gr.Row():
                    with gr.Column(scale=1):
                        gr.Markdown("## Patient Information")
                        patient_id = gr.Textbox(label="Patient ID")
                        age = gr.Number(label="Age", minimum=18, maximum=100)
                        gender = gr.Dropdown(["Male", "Female"], label="Gender")
                        smoking = gr.Dropdown(["Never", "Former", "Current"], label="Smoking Status")

                        gr.Markdown("## Weekly Measurements")
                        daily_inputs = []
                        with gr.Tabs():
                            for day in range(1, 8):
                                with gr.Tab(f"Day {day}"):
                                    fev1 = gr.Number(label=f"FEV1 (Day {day})", minimum=1.0, maximum=5.0)
                                    fvc = gr.Number(label=f"FVC (Day {day})", minimum=1.5, maximum=6.0)
                                    tlc = gr.Number(label=f"TLC (Day {day})", minimum=3.0, maximum=8.0)
                                    vc = gr.Number(label=f"VC (Day {day})", minimum=2.0, maximum=6.0)
                                    daily_inputs.extend([fev1, fvc, tlc, vc])

                    with gr.Column(scale=2):
                        gr.Markdown("## Analysis Results")
                        plot = gr.Plot()
                        report = gr.Markdown()
                        risk = gr.Label()
                        submit = gr.Button("Analyze Health Status", variant="primary", elem_classes=["ripple"])

            with gr.Tab("Database"):
                gr.Markdown("## Patient Records")
                refresh_db = gr.Button("Refresh Database", variant="primary", elem_classes=["ripple"])
                db_output = gr.HTML()

            with gr.Tab("User Management"):
                gr.Markdown("## Add New User")
                new_username = gr.Textbox(label="Username")
                new_password = gr.Textbox(label="Password", type="password")
                new_role = gr.Dropdown(["admin", "doctor"], label="Role")
                add_user_btn = gr.Button("Add User", variant="primary", elem_classes=["ripple"])
                user_message = gr.Markdown("")

    # Event handlers
    login_button.click(
        login,
        inputs=[username, password],
        outputs=[login_block, main_app, login_message]
    )

    logout_btn.click(
        handle_logout,
        inputs=[],
        outputs=[login_block, main_app, login_message]
    )

    all_inputs = [age, gender, smoking, patient_id] + daily_inputs
    submit.click(predict_health, inputs=all_inputs, outputs=[plot, report, risk])
    refresh_db.click(view_database, inputs=[], outputs=[db_output])

    def handle_add_user(username, password, role):
        success, message = add_user(username, password, role)
        if success:
            return gr.update(value=""), gr.update(value=""), gr.update(), f"✅ {message}"
        else:
            return gr.update(), gr.update(), gr.update(), f"❌ {message}"

    add_user_btn.click(
        handle_add_user,
        inputs=[new_username, new_password, new_role],
        outputs=[new_username, new_password, new_role, user_message]
    )

if __name__ == "__main__":
    demo.launch()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/50
150/150 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.9053 - auc: 0.9367 - loss: 0.2224 - val_accuracy: 0.9908 - val_auc: 0.9994 - val_loss: 0.0418
Epoch 2/50
150/150 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9883 - auc: 0.9989 - loss: 0.0332 - val_accuracy: 0.9919 - val_auc: 0.9993 - val_loss: 0.0241
Epoch 3/50
150/150 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9897 - auc: 0.9992 - loss: 0.0286 - val_accuracy: 0.9912 - val_auc: 0.9990 - val_loss: 0.0227
Epoch 4/50
150/150 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9902 - auc: 0.9988 - loss: 0.0296 - val_accuracy: 0.9919 - val_auc: 0.9990 - val_loss: 0.0221
Epoch 5/50
150/150 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9915 - auc: 0.9993 - loss: 0.0249 - val_accuracy: 0.9923 - val_auc: 0.9990 - val_loss: 0.0220
Epoch 6/50
150/150 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9909 - auc: 0.9993 - loss: 0.0266 - val_accuracy: 0.9921 - val_auc: 0.9993 - val_loss: 0.0219
Epoch 7/50
150/150 ━━━━━━━━━━━━━━━━━━━━ 